# Setup Gabon GEDI L4A Testing

1. Download Gabon outline in geospatial format
2. Save to the workspace, probably not the repo?
3. Query CMR with the bbox of the polygon to find out how many granules are involved

Boundary file is available at `shared-buckets/alexdevseed/iso3/GAB-ADM0.json`

In [154]:
import json
import os
import os.path
from typing import Any, Callable, Mapping, Optional, Iterable, TypeVar

import geopandas as gpd
import requests
from shapely.geometry import Polygon
from maap.maap import Granule, MAAP
from profilehooks import timecall

## Get Gabon GeoBoundary

In [160]:
def get_geo_boundary(*, iso: str, level: int) -> gpd.GeoDataFrame:
    file_path = f"/projects/my-public-bucket/iso3/{iso}-ADM{level}.json"

    if not os.path.exists(file_path):
        r = requests.get(
            "https://www.geoboundaries.org/gbRequest.html",
            dict(ISO=iso, ADM=f"ADM{level}"),
        )
        r.raise_for_status()
        dl_url = r.json()[0]["gjDownloadURL"]
        geo_boundary = requests.get(dl_url).json()

        with open(file_path, "w") as out:
            out.write(json.dumps(geo_boundary))

    return gpd.read_file(file_path)


aoi_gdf = get_geo_boundary(iso="GNQ", level=0)
aoi_gdf

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry
0,Equatorial Guinea,GNQ,GNQ-ADM0-3_0_0-B1,GNQ,ADM0,"MULTIPOLYGON (((5.63304 -1.40378, 5.63342 -1.4..."


## Find GEDI L4A Collection

In [161]:
nasa_cmr_host = "cmr.earthdata.nasa.gov"
maap_cmr_host = "cmr.maap-project.org"
maap = MAAP("api.ops.maap-project.org")
cmr_host = maap_cmr_host

gedi_l4a_doi = "10.3334/ORNLDAAC/1986"
gedi_l4a = maap.searchCollection(cmr_host=cmr_host, doi=gedi_l4a_doi, limit=1)[0]
gedi_l4a_concept_id = gedi_l4a["concept-id"]

## Find GEDI L4A Granules within Gabon Bounding Box

In [162]:
# TODO Handle cases where there are more than 2000 granules,
# as 2000 is the largest limit value allowed for a single query
# (a CMR constraint).
granules = timecall(maap.searchGranule)(
    cmr_host=cmr_host,
    collection_concept_id=gedi_l4a_concept_id,
    bounding_box=",".join(map(str, aoi_gdf.total_bounds)),
    # polygon='',
    limit=1200,
)

print(f"Found {len(granules)} granules")


  searchGranule (/maap-py/maap/maap.py:104):
    88.090 seconds



Found 1008 granules


In [163]:
def granules_spatial_filter(granule, aoi):
    # return true or false if granule polygon intersects aoi
    # reading bounding geometries
    polygons = granule["Granule"]["Spatial"]["HorizontalSpatialDomain"]["Geometry"][
        "GPolygon"
    ]["Boundary"]["Point"]
    granule_poly = Polygon(
        [[float(p["PointLongitude"]), float(p["PointLatitude"])] for p in polygons]
    )

    return granule_poly.intersects(aoi)


results = [granules_spatial_filter(g, aoi_gdf.geometry[0]) for g in granules]
filtered_granules = [d for d, s in zip(granules, results) if s]

In [165]:
print(sum(results))
print(sum(results) == len(filtered_granules))

252
True
